In [4]:
# Set up access credentials
consumer_info = [line.strip() for line in open("df_twitter_consumer_auth.txt")]
access_info = [line.strip() for line in open("df_twitter_access_auth.txt")]


In [12]:
# source: https://gist.github.com/yanofsky/5436496

import tweepy #https://github.com/tweepy/tweepy
import csv

#Twitter API credentials
consumer_key = consumer_info[0]
consumer_secret = consumer_info[1]
access_key = access_info[0] 
access_secret = access_info[1]


def get_all_tweets(screen_name):
	#Twitter only allows access to a users most recent 3240 tweets with this method
	
	#authorize twitter, initialize tweepy
	auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
	auth.set_access_token(access_key, access_secret)
	api = tweepy.API(auth)
	
	#initialize a list to hold all the tweepy Tweets
	alltweets = []	
	
	#make initial request for most recent tweets (200 is the maximum allowed count)
	new_tweets = api.user_timeline(screen_name = screen_name,count=200)
	
	#save most recent tweets
	alltweets.extend(new_tweets)
	
	#save the id of the oldest tweet less one
	oldest = alltweets[-1].id - 1
	
	#keep grabbing tweets until there are no tweets left to grab
	while len(new_tweets) > 0:
		print ("getting tweets before %s" % (oldest))
		
		#all subsiquent requests use the max_id param to prevent duplicates
		new_tweets = api.user_timeline(screen_name = screen_name,count=200,max_id=oldest)
		
		#save most recent tweets
		alltweets.extend(new_tweets)
		
		#update the id of the oldest tweet less one
		oldest = alltweets[-1].id - 1
		
		print ("...%s tweets downloaded so far" % (len(alltweets)))
	
	#transform the tweepy tweets into a 2D array that will populate the csv	
	outtweets = [[tweet.id_str, tweet.created_at, tweet.text.encode("utf-8")] for tweet in alltweets]
	
	#write the csv	
	with open('%s_tweets.csv' % screen_name, 'wb') as f:
		writer = csv.writer(f)
		writer.writerow(["id","created_at","text"])
		writer.writerows(outtweets)
	
	pass



In [13]:
get_all_tweets("J_tsar")

getting tweets before 293289746275381247
...384 tweets downloaded so far
getting tweets before 265965404604747775
...579 tweets downloaded so far
getting tweets before 223876232096387071
...775 tweets downloaded so far
getting tweets before 200707976116449279
...967 tweets downloaded so far
getting tweets before 179861681109872640
...1007 tweets downloaded so far
getting tweets before 128727410161102847
...1007 tweets downloaded so far


TypeError: a bytes-like object is required, not 'str'